In [5]:
# import libraries
import json
from elasticsearch import Elasticsearch
from sentence_transformers import SentenceTransformer

#### Step 1: Prepare documents

In [6]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [7]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

documents[1]

{'text': 'GitHub - DataTalksClub/data-engineering-zoomcamp#prerequisites',
 'section': 'General course-related questions',
 'question': 'Course - What are the prerequisites for this course?',
 'course': 'data-engineering-zoomcamp'}

#### Step 2: Create Embeddings using Pretrained Models

In [8]:
model = SentenceTransformer("all-mpnet-base-v2")

In [9]:
len(model.encode("This is a simple sentence"))

768

In [10]:
# pick one to view documents.json data
documents[1]

{'text': 'GitHub - DataTalksClub/data-engineering-zoomcamp#prerequisites',
 'section': 'General course-related questions',
 'question': 'Course - What are the prerequisites for this course?',
 'course': 'data-engineering-zoomcamp'}

In [14]:
# encode the text of one document
test = model.encode(documents[1]['text'])

print(len(test))

768


In [ ]:
# created the dense vector using the pre-trained model
operations = []

for doc in documents:
    # transforming the title into an embedding using the model
    doc["text_vector"] = model.encode(doc["text"]).tolist()

    operations.append(doc)

In [ ]:
operations[1]

{'text': 'GitHub - DataTalksClub/data-engineering-zoomcamp#prerequisites',
 'section': 'General course-related questions',
 'question': 'Course - What are the prerequisites for this course?',
 'course': 'data-engineering-zoomcamp',
 'text_vector': [-0.03641175478696823,
  0.020610174164175987,
  -0.04089024290442467,
  -0.01847684197127819,
  -0.01818760298192501,
  0.016543572768568993,
  -0.008730770088732243,
  -0.009902060963213444,
  0.029097313061356544,
  0.012701984494924545,
  0.026511475443840027,
  -0.01796942763030529,
  -0.0016375882551074028,
  0.11685211956501007,
  0.02934090606868267,
  -0.025640197098255157,
  0.026349829509854317,
  0.02825084701180458,
  -0.08533276617527008,
  -0.001631500432267785,
  -0.013839669525623322,
  0.008223621174693108,
  -0.023533973842859268,
  -0.03795081004500389,
  0.0275421142578125,
  0.016090914607048035,
  -0.021919315680861473,
  0.013490717858076096,
  -0.016174105927348137,
  -0.011982636526226997,
  0.026945536956191063,
  0

#### Step 3: Setup ElasticSearch connection

In [17]:
es_client = Elasticsearch('http://localhost:9200') 
es_client.info()

ObjectApiResponse({'name': '2ad6708d9512', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'jqbq6mYdTkqcCCi9Nyf57A', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

#### Step 4: Create Mappings and Index

In [22]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} ,
            "text_vector": {"type": "dense_vector", "dims": 768, "index": True, "similarity": "cosine"},
        }
    }
}

In [24]:
index_name = "course-questions"

es_client.indices.delete(index = index_name, ignore_unavailable = True)
es_client.indices.create(index = index_name, body = index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

#### Step 5: Add documents into index

In [25]:
for doc in operations:
    try:
        es_client.index(index = index_name, document = doc)
    except Exception as e:
        print(e)

#### Step 6: Create end user query

In [26]:
search_term = "windows or mac?"
vector_search_term = model.encode(search_term)

In [27]:
query = {
    "field": "text_vector", # the field in the Elasticsearch index that stores the vector data
    "query_vector": vector_search_term, # the vector want to search for, typically the result of embedding a search term
    "k": 5, # the number of nearest neighbors (top k) to return in the search results
    "num_candidates": 10000, # the number of candidate vectors to consider when performing the search, help optimize search performance.
}

In [28]:
res = es_client.search(
    index = index_name, 
    knn = query,
    source = ["text", "section", "question", "course"])

res["hits"]["hits"]

[{'_index': 'course-questions',
  '_id': 'POALxZMBWF1fNzS8cFtF',
  '_score': 0.71479195,
  '_source': {'question': 'Environment - Is the course [Windows/mac/Linux/...] friendly?',
   'course': 'data-engineering-zoomcamp',
   'section': 'General course-related questions',
   'text': 'Yes! Linux is ideal but technically it should not matter. Students last year used all 3 OSes successfully'}},
 {'_index': 'course-questions',
  '_id': 'XuAMxZMBWF1fNzS8Ll7C',
  '_score': 0.6107274,
  '_source': {'question': 'WSL instructions',
   'course': 'mlops-zoomcamp',
   'section': 'Module 1: Introduction',
   'text': 'If you wish to use WSL on your windows machine, here are the setup instructions:\nCommand: Sudo apt install wget\nGet Anaconda download address here. wget <download address>\nTurn on Docker Desktop WFree Download | AnacondaSL2\nCommand: git clone <github repository address>\nVSCODE on WSL\nJupyter: pip3 install jupyter\nAdded by Gregory Morris (gwm1980@gmail.com)\nAll in all softwares a

#### Step 7: Perform Keyword search with Semantic Search (Hybrid/ Advanced Search)

In [29]:
knn_query = {
    "field": "text_vector",
    "query_vector": vector_search_term,
    "k": 5,
    "num_candidates": 10000
}

In [30]:
response = es_client.search(
    index = index_name,
    query = {
        "match": {"section": "General course-related questions"},
    }, # keyword search
    knn = knn_query, # semantic search
    size = 5 # number of result will return
)

response["hits"]["hits"]

[{'_index': 'course-questions',
  '_id': 'POALxZMBWF1fNzS8cFtF',
  '_score': 11.598243,
  '_source': {'text': 'Yes! Linux is ideal but technically it should not matter. Students last year used all 3 OSes successfully',
   'section': 'General course-related questions',
   'question': 'Environment - Is the course [Windows/mac/Linux/...] friendly?',
   'course': 'data-engineering-zoomcamp',
   'text_vector': [-0.026965467259287834,
    -0.0006261487724259496,
    -0.016629496589303017,
    0.05285152047872543,
    0.054765284061431885,
    -0.031339891254901886,
    0.029942581430077553,
    -0.04808562994003296,
    0.04467552527785301,
    0.0058394744992256165,
    0.016233038157224655,
    0.012001179158687592,
    -0.03122226521372795,
    0.016600513830780983,
    -0.04886903986334801,
    -0.06496306508779526,
    0.046434223651885986,
    -0.009297751821577549,
    -0.0642528384923935,
    -0.01373270247131586,
    -0.01597621664404869,
    0.008629538118839264,
    -0.02447898499